In [15]:
import sys
from pathlib import Path

SRC = Path().resolve() / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch
import torch.nn as nn

In [17]:
from device_utils import get_device
device = get_device()
print("Using device:", device)

Using device: mps


In [18]:
dataset_usage = 1
dataset_regenerate = False

seq_size = 32 # число токеном в скользящем окне датасета
seq_stride = 1

emb_dim=256
hidden_dim=512
num_layers=2
dropout_p=0.3
learning_rate=1e-3
weight_decay=0.01

model_train = True
model_load_path = "models/next_token_16_250822_212717.pth"

In [19]:
from data_utils import process_dataset, read_splits, truncate_ratio_and_clear

if dataset_regenerate:
    process_dataset("data/raw_dataset.csv")

train_df, val_df, test_df = read_splits("data/dataset_processed.csv")

train_texts = truncate_ratio_and_clear(list(train_df['text']), dataset_usage)
val_texsts = truncate_ratio_and_clear(list(val_df['text']), dataset_usage)
test_texts = truncate_ratio_and_clear(list(test_df['text']), dataset_usage)

print(len(train_texts))

1277123


In [20]:
from transformers import BertTokenizerFast
from tokenizer_utils import resolve_eos_token_id

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

eos_token_id = resolve_eos_token_id(tokenizer)

print("eos_token_id:", eos_token_id)

eos_token_id: 102


In [21]:

from torch.utils.data import DataLoader
from next_token_dataset import NextTokenDataset

train_dataset = NextTokenDataset(
    train_texts, 
    tokenizer, 
    eos_id=eos_token_id, 
    seq_size=seq_size, 
    stride=seq_stride
)

val_dataset = NextTokenDataset(
    val_texsts,
    tokenizer,
    eos_id=eos_token_id,
    seq_size=seq_size,
    stride=seq_stride
)

print(f"Train samples count: {len(train_dataset)}")
print(f"Val samples count: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

window: 100%|██████████| 2504159/2504159 [00:01<00:00, 1591329.46it/s]

Train samples count: 20079790
Val samples count: 2504159


In [22]:
from lstm_model import NextTokenLSTM

model = NextTokenLSTM(
    vocab_size=tokenizer.vocab_size,
    emb_dim=emb_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout_p=dropout_p,
)

In [23]:
from lstm_train import train_next_token
from datetime import datetime

if model_train:
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    model = train_next_token(
        model,
        optimizer=optimizer,
        criterion=criterion,
        tokenizer=tokenizer,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        epochs=5
    )

    timestamp = datetime.now().strftime("%y%m%d_%H%M%S")
    torch.save(model.state_dict(), f"models/next_token_{seq_size}_{timestamp}.pth")

Epoch: 1:   0%|          | 567/156874 [01:36<7:21:37,  5.90it/s]


KeyboardInterrupt: 

In [ ]:
if not model_train:
    state = torch.load(model_load_path, map_location=device)
    model.load_state_dict(state)

In [ ]:
prompts = [
    "I really want to",
    "Can you please tell me",
    "The main reason is",
    "When I was a child",
    "I think it would be better",
    "Let's go to the",
    "It is important because",
    "In the future I would like to",
    "Do you know how to",
    "One of my favorite things"
]

In [ ]:
from autocomplete import autocomplete_text

for prompt in prompts:
    comp_text = autocomplete_text(
        model=model, 
        tokenizer=tokenizer,
        eos_id=eos_token_id,
        seq_size=seq_size,
        text=prompt,
    )
    print(f"{prompt} -> {comp_text}")
    print("=" * 50)

I really want to -> go to the gym but i m not sure i can get it to work tomorrow i m going to miss my friends and i m going to miss my friends and i m going to miss my friends and i m going to miss my friends and i m
Can you please tell me -> what you think of the quot i m a celebrity quot i m not sure i m going to be able to go to the show tonight i m going to miss my friends and i m going to miss my friends and i m going to miss
The main reason is -> i m not a fan of the twitterverse i m not sure i m going to be able to get my phone back to my phone and i m not sure i can get it to work tomorrow i m going to miss my friends and i m going
When I was a child -> i m not sure i m going to be able to go to the gym today i m going to miss my baby i m so excited for the new moon trailor i m so excited i m gonna miss the season finale of the hills season finale of
I think it would be better -> if i had a good time i m going to be a big fan of yours and i m not sure i can make it to the show 

In [ ]:
from eval_transformer_pipeline import build_gpt2_pipeline_fn

gpt2_generate, tok = build_gpt2_pipeline_fn(
    model_name="distilgpt2",
    max_new_tokens=32,
    temperature=0.8,
    top_p=0.95,
    device=device
)

for prompt in prompts:
    comp_text = gpt2_generate(prompt)
    print(f"{prompt} -> {comp_text}")
    print("=" * 50)

Device set to use mps


I really want to -> come out here and talk about things, and I want to be a part of the story, too."
Can you please tell me -> when it is time to get on board or start running your own business!
The main reason is -> not to see a single black woman and then to see a man with a white hair, but to see a woman with a white hair. The same principle applies
When I was a child -> , I was scared that I would be left alone. So I went to my parents' house and started to take care of my young daughter and I would give
I think it would be better -> for the community," said Mr. Shumipan.
Let's go to the -> game and hit the bottom of the screen and watch all the moves and progress. Then it's back in the game and it's all over again.
It is important because -> it is possible for people to think, to be happy and happy and to be healthy.”





This article is part of the
In the future I would like to -> thank the people who gave me an amazing interview and the people that gave me a wonderful interview

In [ ]:
from rouge_autocomplete_tester import PairingConfig, RougeAutocompleteTester

def lstm_generate(prompt: str) -> str:
    return autocomplete_text(
        model=model, 
        tokenizer=tokenizer,
        eos_id=eos_token_id,
        seq_size=seq_size,
        text=prompt,
    )

tester = RougeAutocompleteTester(
    tokenizer=tok,
    pairing=PairingConfig(prompt_len=16, cont_len=32, max_text_tokens=256),
    use_stemmer=True
)

print("GPT2 Transformer ROUGE Testing:")
gpt_result = tester.evaluate(test_texts[:1000], generator_fn=gpt2_generate, limit_pairs=None)
print(gpt_result)

print("=" * 50)

print("RRN LSTM ROUGE Testing:")
rnn_result = tester.evaluate(test_texts[:1000], generator_fn=lstm_generate, limit_pairs=None)
print(rnn_result)

GPT2 Transformer ROUGE Testing:


 54%|█████▎    | 221/413 [01:24<01:13,  2.61it/s]


KeyboardInterrupt: 

## Подходы к автокомплиту
- **RNN (LSTM):**  
  Использован рекуррентный метод. На каждом шаге из выходной последовательности берутся логиты, соответствующие последнему токену, из них с помощью `argmax` выбирается наиболее вероятный токен, который затем подставляется в конец входной последовательности на следующем шаге.

- **Transformer (GPT-2):**  
  Использован стандартный autoregressive-подход модели `generate`, где последовательность продолжается на основе ранее сгенерированных токенов.

## Метрики
Для оценки использовалась метрика **ROUGE**:

1. **Во время обучения:**  
   Сравнение предсказаний по скользящему окну (teacher forcing).

2. **На тестовых данных:**  
   Тексты делились на две части:
   - `prompt` — начало текста.  
   - `continuation` — эталонное продолжение.  
   Считался ROUGE между сгенерированным продолжением и эталонным.

Метрика ROUGE по разбиению текстов на 2 части применялась как для RNN, так и для GPT-2.

## Результаты
- Результаты оказались **сравнимыми**.  
- Transformer показал **немного лучшие значения ROUGE**, что объясняется предварительным обучением на текстах Twitter и лучшим предсказанием таких же текстов.  
- **RNN работает значительно быстрее**, особенно на ограниченных ресурсах.  
- **Качество текста:**
  - GPT-2 генерирует более богатые и осмысленные тексты.  
  - LSTM даёт сухие ответы и часто не попадает в контекст.

## Выводы
- Для простого автокомплита, где **не требуется глубокий контекст** и важна работа на мобильных устройствах с ограниченной памятью и вычислительными ресурсами, больше подходит **RNN (LSTM)**.  
- Для более содержательных и осмысленных продолжений текста лучше использовать **Transformer (GPT-2)**.